### Code to connect to Oracle from our local python notebook. 
####This was used in our initial attempt to make Oracle work with pandas before we ran into the memory issues:

import os

import platform

LOCATION = r"C:\Oracle"

os.environ["PATH"] = LOCATION + ";" + os.environ["PATH"]

import cx_Oracle

dsn_tns = cx_Oracle.makedsn('msb-MIS381N.austin.utexas.edu', '1521', sid='orcl') #if needed, place an 'r' before any parameter in order to address any special character such as '\'. 
conn = cx_Oracle.connect(user=r'JAR8892', password='<password to be entered here>', dsn=dsn_tns) #if needed, place an 'r' before any parameter in order to address any special character such as '\'. For example, if your user name contains '\', you'll need to place 'r' before the user name: user=r'User Name'

c = conn.cursor()

c.execute('select * from JAR8892.CATEGORIES') # use triple quotes if you want to spread your query across multiple lines

for row in c:
  print (row[0], '-', row[1]) # this only shows the first two columns. To add an additional column you'll need to add , '-', row[2], etc.

conn.close()

### Code for our Recommendation engine (Existing User)

In [3]:
%sh pip install implicit

Collecting implicit
 Downloading https://files.pythonhosted.org/packages/5a/d8/6b4f1374ffa2647b72ac76960c71b984c6f3238090359fb419d03827d87a/implicit-0.4.2.tar.gz (1.1MB)
Requirement already satisfied: numpy in /databricks/python3/lib/python3.7/site-packages (from implicit) (1.16.2)
Requirement already satisfied: scipy>=0.16 in /databricks/python3/lib/python3.7/site-packages (from implicit) (1.2.1)
Collecting tqdm>=4.27 (from implicit)
 Downloading https://files.pythonhosted.org/packages/a5/13/cd55c23e3e158ed5b87cae415ee3844fc54cb43803fa3a0a064d23ecb883/tqdm-4.40.0-py2.py3-none-any.whl (54kB)
Building wheels for collected packages: implicit
 Building wheel for implicit (setup.py): started
 Building wheel for implicit (setup.py): still running...
 Building wheel for implicit (setup.py): finished with status 'done'
 Stored in directory: /root/.cache/pip/wheels/1b/48/b1/1aebe3acc3afb5589e72d3e7c3ffc3f637dc4721c1a974dff7
Successfully built implicit
Installing collected packages: tqdm, implicit
Successfully installed implicit-0.4.2 tqdm-4.40.0
You are using pip version 19.0.3, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

In [4]:
dbutils.widgets.removeAll()

In [5]:
user_id = getArgument("Enter your User ID ", "TEXTBOX_STRING")
city = getArgument("Enter your City (San Francisco, New York or Chicago)","TEXTBOX_STRING")

Deprecation warning: Use dbutils.widgets.text() or dbutils.widgets.dropdown() to create a widget and dbutils.widgets.get() to get its bound value.
Deprecation warning: Use dbutils.widgets.text() or dbutils.widgets.dropdown() to create a widget and dbutils.widgets.get() to get its bound value.

In [6]:
import os
import platform

import implicit
from scipy.sparse import coo_matrix
from scipy.sparse import csr_matrix
import numpy as np
import pandas as pd

Reading in members file to a spark dataframe

In [8]:
df_members = spark\
  .read\
  .option("inferSchema", "true")\
  .option("header", "true")\
  .csv("/FileStore/tables/members.csv")

In [9]:
# selecting required columns and converting t0 pandas dataframe
df_members = df_members.select('member_id', 'city','group_id').toPandas()
df_members.head()

,member_id,city,group_id
0,3,New York,490552
1,3,New York,1474611
2,3,New York,1490492
3,3,New York,1515830
4,3,New York,1574965


Reading in groups file to a spark dataframe

In [11]:
df_groups_temp = spark\
  .read\
  .option("inferSchema", "true")\
  .option("header", "true")\
  .csv("/FileStore/tables/groups.csv")

df_groups_temp.head()

Out[10]: Row(group_id=6388, category_id=14, category.name='health/wellbeing', category.shortname='health-wellbeing', city_id=10001, city='New York', country='US', created=datetime.datetime(2002, 11, 21, 16, 50, 46), description='Those who practice or hold a strong interest in the many fields of alternative medicine and health.', group_photo.base_url='https://secure.meetupstatic.com', group_photo.highres_link='https://secure.meetupstatic.com/photos/event/1/d/3/a/highres_61087482.jpeg', group_photo.photo_id=61087482, group_photo.photo_link='https://secure.meetupstatic.com/photos/event/1/d/3/a/600_61087482.jpeg', group_photo.thumb_link='https://secure.meetupstatic.com/photos/event/1/d/3/a/thumb_61087482.jpeg', group_photo.type='event', join_mode='open', lat=40.75, link='https://www.meetup.com/alternative-health-nyc/', lon=-73.989998, members=1440, group_name='Alternative Health NYC', organizer.member_id=1513133, organizer.name='Joel E.', organizer.photo.base_url='https://secure.meetupstatic.com', organizer.photo.highres_link='https://secure.meetupstatic.com/photos/member/5/9/b/0/highres_3982960.jpeg', organizer.photo.photo_id=3982960, organizer.photo.photo_link='https://secure.meetupstatic.com/photos/member/5/9/b/0/member_3982960.jpeg', organizer.photo.thumb_link='https://secure.meetupstatic.com/photos/member/5/9/b/0/thumb_3982960.jpeg', organizer.photo.type='member', rating=4.39, state='NY', timezone='US/Eastern', urlname='alternative-health-nyc', utc_offset=-14400, visibility='public', who='Explorers of Health')

In [12]:
# selecting required columns and converting to pandas dataframe

df_groups_temp = df_groups_temp.select('category_id','group_id','city','group_name','members', 'rating').toPandas()
df_groups_temp.head()

,category_id,group_id,city,group_name,members,rating
0,14,6388,New York,Alternative Health NYC,1440,4.39
1,4,6510,New York,Alternative Energy Meetup,969,4.31
2,26,8458,New York,NYC Animal Rights,2930,4.84
3,29,8940,New York,The New York City Anime Group,5080,4.46
4,26,10104,New York,NYC Pit Bull Group,2097,4.09


Loading categories list to a spark dataframe.

In [14]:
df_categories = spark\
  .read\
  .option("inferSchema", "true")\
  .option("header", "true")\
  .csv("/FileStore/tables/categories.csv")

df_categories.head()

Out[12]: Row(category_id=1, category_name='Arts & Culture', shortname='Arts', sort_name='Arts & Culture')

In [15]:
#merging categories to groups
df_categories = df_categories.select('category_id','category_name').toPandas()
df_groups = pd.merge(df_groups_temp, df_categories, on='category_id')
df_groups.head()

,category_id,group_id,city,group_name,members,rating,category_name
0,14,6388,New York,Alternative Health NYC,1440,4.39,Health & Wellbeing
1,14,54126,New York,Energy Healers NYC,2027,4.83,Health & Wellbeing
2,14,67776,New York,Flourishing Life Meetup,839,4.56,Health & Wellbeing
3,14,111855,New York,Hypnosis & NLP NYC - Update Your Brain,1403,4.69,Health & Wellbeing
4,14,129277,Chicago,The Live Food Chicago Community,1997,4.77,Health & Wellbeing


Function that provides a subset of the members and groups dataframe pertaining to that city

In [17]:
def get_city(city):
    """Take in the acronym of the city and return a dataframe containing only members and groups from that city."""
    
    #defense coding
    if city not in ['SF', 'NYC', 'CHI']:
        raise ValueError('Invalid City')
        
    #separating the members and groups by their corresponding city
   
    #SF
    sf = ['San Francisco', 'san francisco', 'South San Francisco']
    df_sf = df_members[df_members.city.isin(sf)]
    df_sfg = df_groups[df_groups.city.isin(sf)]
    
    #NY
    ny = ['New York', 'West New York', 'New York Mills']
    df_ny = df_members[df_members.city.isin(ny)]
    df_nyg = df_groups[df_groups.city.isin(ny)]
    
    #CHI
    chi = ['Chicago','Chicago Heights','West Chicago','Chicago Ridge','East Chicago','North Chicago','Chicago Park']
    df_chi = df_members[df_members.city.isin(chi)]
    df_chig = df_groups[df_groups.city.isin(chi)]
    
    if city == 'SF':
        return df_sf.merge(df_sfg, on = 'group_id')
    elif city == 'NYC':
        return df_ny.merge(df_nyg, on = 'group_id')
    elif city == 'CHI':
        return df_chi.merge(df_chig, on = 'group_id')

ALS Model training for SF

In [19]:
  df1 = get_city('SF')
  df1['m_code'] = df1['member_id'].astype('category').cat.codes
  df1['g_code'] = df1['group_id'].astype('category').cat.codes
  df1['dummy'] = 1

  print(df1.head())
  
  #create the 2 separate sparse matrix where 1 is item x user and the other user x item
  item_user1 = csr_matrix((df1['dummy'], (df1['g_code'],df1['m_code']))) #required to fit the model
  user_item1 = csr_matrix((df1['dummy'], (df1['m_code'], df1['g_code']))) #used as an input for recommend method
  
  #initializing the ALS model
  model1 = implicit.als.AlternatingLeastSquares(factors = 20, regularization = .01, iterations = 20)

  #preset alpha for the confidence learning rate
  alpha = 15

  #confidence c is defined by 1 + alpha*item_user where item_user is the interaction between an item and user
  #Note: the model takes care of the 1 so any negative preference will be set to a confidence of 1 when fitting
  c_item_user1 = item_user1 * alpha

  #fitting the model using the item_user matrix after it has been multiplied by alpha
  model1.fit(c_item_user1)

member_id city_x group_id ... m_code g_code dummy
0 65 San Francisco 2701562 ... 0 583 1
1 29009 San Francisco 2701562 ... 38 583 1
2 67424 San Francisco 2701562 ... 58 583 1
3 94707 San Francisco 2701562 ... 83 583 1
4 95826 San Francisco 2701562 ... 84 583 1

[5 rows x 12 columns]
WARNING:root:OpenBLAS detected. Its highly recommend to set the environment variable 'export OPENBLAS_NUM_THREADS=1' to disable its internal multithreading
 0%| | 0/20 [00:00<?, ?it/s] 5%|▌ | 1/20 [00:03<01:14, 3.94s/it] 10%|█ | 2/20 [00:08<01:11, 3.99s/it] 15%|█▌ | 3/20 [00:12<01:11, 4.20s/it] 20%|██ | 4/20 [00:16<01:06, 4.14s/it] 25%|██▌ | 5/20 [00:20<01:01, 4.12s/it] 30%|███ | 6/20 [00:25<00:59, 4.22s/it] 35%|███▌ | 7/20 [00:29<00:56, 4.35s/it] 40%|████ | 8/20 [00:34<00:52, 4.39s/it] 45%|████▌ | 9/20 [00:38<00:47, 4.34s/it] 50%|█████ | 10/20 [00:42<00:42, 4.29s/it] 55%|█████▌ | 11/20 [00:46<00:38, 4.23s/it] 60%|██████ | 12/20 [00:50<00:33, 4.18s/it] 65%|██████▌ | 13/20 [00:54<00:28, 4.12s/it] 70%|███████ | 14/20 [00:58<00:24, 4.10s/it] 75%|███████▌ | 15/20 [01:02<00:20, 4.02s/it] 80%|████████ | 16/20 [01:06<00:16, 4.06s/it] 85%|████████▌ | 17/20 [01:11<00:12, 4.11s/it] 90%|█████████ | 18/20 [01:15<00:08, 4.10s/it] 95%|█████████▌| 19/20 [01:19<00:04, 4.15s/it]100%|██████████| 20/20 [01:23<00:00, 4.12s/it]100%|██████████| 20/20 [01:23<00:00, 4.18s/it]

ALS Model training for New York

In [21]:
  df2 = get_city('NYC')
  df2['m_code'] = df2['member_id'].astype('category').cat.codes
  df2['g_code'] = df2['group_id'].astype('category').cat.codes
  df2['dummy'] = 1
  
  print(df2.head())
  
  #create the 2 separate sparse matrix where 1 is item x user and the other user x item
  item_user2 = csr_matrix((df2['dummy'], (df2['g_code'],df2['m_code']))) #required to fit the model
  user_item2 = csr_matrix((df2['dummy'], (df2['m_code'], df2['g_code']))) #used as an input for recommend method
  
  #initializing the ALS model
  model2 = implicit.als.AlternatingLeastSquares(factors = 20, regularization = .01, iterations = 20)

  #preset alpha for the confidence learning rate
  alpha = 15

  #confidence c is defined by 1 + alpha*item_user where item_user is the interaction between an item and user
  #Note: the model takes care of the 1 so any negative preference will be set to a confidence of 1 when fitting
  c_item_user2 = item_user2 * alpha

  #fitting the model using the item_user matrix after it has been multiplied by alpha
  model2.fit(c_item_user2)

member_id city_x group_id ... m_code g_code dummy
0 3 New York 490552 ... 0 333 1
1 210 New York 490552 ... 5 333 1
2 357518 New York 490552 ... 1086 333 1
3 511008 New York 490552 ... 1722 333 1
4 915553 New York 490552 ... 2981 333 1

[5 rows x 12 columns]
 0%| | 0/20 [00:00<?, ?it/s] 5%|▌ | 1/20 [00:08<02:42, 8.56s/it] 10%|█ | 2/20 [00:16<02:33, 8.50s/it] 15%|█▌ | 3/20 [00:25<02:24, 8.53s/it] 20%|██ | 4/20 [00:34<02:16, 8.53s/it] 25%|██▌ | 5/20 [00:42<02:08, 8.54s/it] 30%|███ | 6/20 [00:51<02:00, 8.60s/it] 35%|███▌ | 7/20 [00:59<01:51, 8.61s/it] 40%|████ | 8/20 [01:08<01:43, 8.66s/it] 45%|████▌ | 9/20 [01:17<01:35, 8.64s/it] 50%|█████ | 10/20 [01:25<01:25, 8.59s/it] 55%|█████▌ | 11/20 [01:34<01:16, 8.53s/it] 60%|██████ | 12/20 [01:42<01:08, 8.57s/it] 65%|██████▌ | 13/20 [01:51<01:00, 8.58s/it] 70%|███████ | 14/20 [02:00<00:51, 8.61s/it] 75%|███████▌ | 15/20 [02:08<00:42, 8.59s/it] 80%|████████ | 16/20 [02:17<00:34, 8.59s/it] 85%|████████▌ | 17/20 [02:26<00:25, 8.65s/it] 90%|█████████ | 18/20 [02:34<00:17, 8.65s/it] 95%|█████████▌| 19/20 [02:43<00:08, 8.59s/it]100%|██████████| 20/20 [02:51<00:00, 8.62s/it]100%|██████████| 20/20 [02:51<00:00, 8.59s/it]

ALS Model training for Chicago

In [23]:
  df3 = get_city('CHI')
  df3['m_code'] = df3['member_id'].astype('category').cat.codes
  df3['g_code'] = df3['group_id'].astype('category').cat.codes
  df3['dummy'] = 1
  
  print(df3.head())
  
  #create the 2 separate sparse matrix where 1 is item x user and the other user x item
  item_user3 = csr_matrix((df3['dummy'], (df3['g_code'],df3['m_code']))) #required to fit the model
  user_item3 = csr_matrix((df3['dummy'], (df3['m_code'], df3['g_code']))) #used as an input for recommend method

    #initializing the ALS model
  model3 = implicit.als.AlternatingLeastSquares(factors = 20, regularization = .01, iterations = 20)

  #preset alpha for the confidence learning rate
  alpha = 15

  #confidence c is defined by 1 + alpha*item_user where item_user is the interaction between an item and user
  #Note: the model takes care of the 1 so any negative preference will be set to a confidence of 1 when fitting
  c_item_user3 = item_user3 * alpha

  #fitting the model using the item_user matrix after it has been multiplied by alpha
  model3.fit(c_item_user3)

member_id city_x group_id ... m_code g_code dummy
0 819 Chicago 514628 ... 0 103 1
1 4295 Chicago 514628 ... 3 103 1
2 10600 Chicago 514628 ... 14 103 1
3 152870 Chicago 514628 ... 101 103 1
4 300185 Chicago 514628 ... 220 103 1

[5 rows x 12 columns]
 0%| | 0/20 [00:00<?, ?it/s] 5%|▌ | 1/20 [00:03<01:07, 3.53s/it] 10%|█ | 2/20 [00:07<01:04, 3.57s/it] 15%|█▌ | 3/20 [00:10<01:00, 3.54s/it] 20%|██ | 4/20 [00:14<00:56, 3.52s/it] 25%|██▌ | 5/20 [00:17<00:53, 3.56s/it] 30%|███ | 6/20 [00:21<00:49, 3.52s/it] 35%|███▌ | 7/20 [00:24<00:45, 3.50s/it] 40%|████ | 8/20 [00:28<00:43, 3.63s/it] 45%|████▌ | 9/20 [00:32<00:39, 3.59s/it] 50%|█████ | 10/20 [00:35<00:35, 3.52s/it] 55%|█████▌ | 11/20 [00:39<00:31, 3.55s/it] 60%|██████ | 12/20 [00:42<00:27, 3.48s/it] 65%|██████▌ | 13/20 [00:45<00:24, 3.48s/it] 70%|███████ | 14/20 [00:49<00:21, 3.52s/it] 75%|███████▌ | 15/20 [00:52<00:17, 3.47s/it] 80%|████████ | 16/20 [00:56<00:14, 3.54s/it] 85%|████████▌ | 17/20 [00:59<00:10, 3.48s/it] 90%|█████████ | 18/20 [01:03<00:06, 3.49s/it] 95%|█████████▌| 19/20 [01:06<00:03, 3.51s/it]100%|██████████| 20/20 [01:10<00:00, 3.45s/it]100%|██████████| 20/20 [01:10<00:00, 3.51s/it]

In [24]:
#this confirms that there isnt any members in sf that isnt in a group
print(df1['group_id'].dropna().shape)
print(df2['group_id'].dropna().shape)
print(df3['group_id'].dropna().shape)

(1472925,)
(3144756,)
(1191548,)

Function to recommend.  Gets the dummy_code of the user and city as inputs

In [26]:
def recommend(code_user_id, user_item, city):

  if city == 'san francisco':
    
    recommend = model1.recommend(code_user_id, user_item, N = 10)
    #unpacking recommend (list of tuples) into their own list
    idx = []
    scores = []
    for item in recommend:
        item_id, score = item
        idx.append(item_id)
        scores.append(score)

    #Showing the recommended group names and corresponding categories
    rec = list(df1[df1['g_code'].isin(idx)].loc[:,'group_name'].unique())
    print(df1[df1['g_code'].isin(idx)].loc[:,['group_name','category_name']].drop_duplicates(subset=['group_name', 'category_name']))
    
    return rec

  elif city == 'new york':
      
    recommend = model2.recommend(code_user_id, user_item, N = 10)
    #unpacking recommend (list of tuples) into their own list
    idx = []
    scores = []
    for item in recommend:
        item_id, score = item
        idx.append(item_id)
        scores.append(score)

      #Showing the recommended group names and corresponding categories
    rec = list(df2[df2['g_code'].isin(idx)].loc[:,'group_name'].unique())
    print(df2[df2['g_code'].isin(idx)].loc[:,['group_name','category_name']].drop_duplicates(subset=['group_name', 'category_name']))
    
    return rec

  elif city == 'chicago':
      
    recommend = model3.recommend(code_user_id, user_item, N = 10)
    #unpacking recommend (list of tuples) into their own list
    idx = []
    scores = []
    for item in recommend:
        item_id, score = item
        idx.append(item_id)
        scores.append(score)

      #Showing the recommended group names and corresponding categories
    rec = list(df3[df3['g_code'].isin(idx)].loc[:,'group_name'].unique())
    print(df3[df3['g_code'].isin(idx)].loc[:,['group_name','category_name']].drop_duplicates(subset=['group_name', 'category_name']))
    
    return rec

  else:
    print("Invalid City Name")
  

Calls the recommend function after checking the city of that the user inputed and display the top 10 recommendations.

In [28]:
city = city.lower() #converts city to lower case
user_id = int(user_id) # converts the user id input to integer type

if city == 'san francisco':
  if (df1['member_id']==user_id).sum()>0:
    print("Existing User\n")
    print("\n Top 10 Recommendations\n")
    code_user_id = df1[df1['member_id']==user_id]['m_code'].iloc[0]
    rec = recommend(code_user_id, user_item1, city)
  
elif city == 'new york':
  if (df2['member_id']==user_id).sum()>0:
    print("Existing User\n")
    print("\n Top 10 Recommendations\n")
    code_user_id = df2[df2['member_id']==user_id]['m_code'].iloc[0]
    rec = recommend(code_user_id, user_item2, city)
  
elif city == 'chicago':
  if (df3['member_id']==user_id).sum()>0:
    print("Existing User\n")
    print("\n Top 10 Recommendations\n")
    code_user_id = df3[df3['member_id']==user_id]['m_code'].iloc[0]
    rec = recommend(code_user_id, user_item3, city)

else:
  print("Invalid City Name")
  

Existing User


 Top 10 Recommendations

 group_name category_name
778 Chicago JavaScript Meetup Group Tech
10139 Chicago HTML5 Tech
15241 Geekfest Tech
34384 Refresh Chicago Tech
67516 Chicago Women Developers Tech
207977 Girl Develop It Chicago Tech
371592 The Chicago Running Meetup Group Fitness
499573 ChicagoRuby Tech
545904 Tech in Motion: Chicago Tech
699451 Chicago WomenTech Tech

Following prints the existing groups of the user

In [30]:
if city == 'san francisco':
  
  #this confirms the recommended groups are not groups that a user is already participating in
  for i in rec:
      if i in df1[df1['member_id'] == user_id]['group_name']:
          print(i)
        
  #groups that this user is in
  
  print(df1[df1['member_id'] == user_id][['group_name','category_name']])
  
elif city == 'new york':

  #this confirms the recommended groups are not groups that a user is already participating in
  for i in rec:
      if i in df1[df1['member_id'] == user_id]['group_name']:
          print(i)
        
  #groups that this user is in
  
  print(df2[df2['member_id'] == user_id][['group_name','category_name']])
  
elif city == 'chicago':
  
  #this confirms the recommended groups are not groups that a user is already participating in
  for i in rec:
      if i in df1[df1['member_id'] == user_id]['group_name']:
          print(i)
        
  #groups that this user is in
  
  print(df3[df3['member_id'] == user_id][['group_name','category_name']])

else:
  print("Invalid City Name")


group_name category_name
40096 Built in Chicago Tech
180037 Refactor::Chicago Tech
233898 Adobe Designers and Developers (AUG Chicago) Tech
252445 Chicago Beer Runners Fitness
254166 Devoxx4Kids Chicagoland Area Tech

### Code for New Users: 
###### Part 1: Recommend 10 groups based on the top 3 category choices.
###### The top 5 groups are the 5 groups from category 1 with the most members, next 3 are top 3 groups from category 2 with the most members and the last 2 groups are the top 2 groups from category 3 with the most members in that category

In [32]:
# To test for different users, the city and category_x variables can be manually changed with values to show different recommendations from various cold starts. Sample values have been input and displayed below:

# Please enter your choices in the below fields where we have entered sample values
city = "New York"
city_mask = df_groups['city']==city
city1 = df_groups[city_mask] 
print("Please select your top three choices from the below list of category options in your city: \n")
for ind,i in enumerate(city1['category_name'].unique()):
  print (ind+1, i)

Please select your top three choices from the below list of category options in your city: 

1 Health & Wellbeing
2 Community & Environment
3 Pets & Animals
4 Sci-Fi & Fantasy
5 Language & Ethnic Identity
6 New Age & Spirituality
7 Religion & Beliefs
8 Movements & Politics
9 Socializing
10 Outdoors & Adventure
11 Dancing
12 Support
13 Games
14 Book Clubs
15 Sports & Recreation
16 Food & Drink
17 Career & Business
18 Photography
19 Music
20 Education & Learning
21 Cars & Motorcycles
22 Tech
23 Parents & Family
24 Movies & Film
25 Hobbies & Crafts
26 Paranormal
27 Writing
28 Arts & Culture
29 Fitness
30 Singles
31 Lifestyle
32 LGBT
33 Fashion & Beauty

In [33]:
# Copy and paste your top 3 choices from the above options into the samples shown below:
cat1 = df_groups['category_name']=='Tech'
cat2 = df_groups['category_name']=='Career & Business'
cat3 = df_groups['category_name']=='Socializing'

In [34]:
print("Your top 10 recommendations based on the above chosen categories are:")
for ind,i in enumerate(df_groups.loc[city_mask&cat1].sort_values(by='members',ascending=False)['group_name'].head(5).values):
  print (ind+1, i)
for ind,i in enumerate(df_groups.loc[city_mask&cat2].sort_values(by='members',ascending=False)['group_name'].head(3).values):
  print (ind+6, i)
for ind,i in enumerate(df_groups.loc[city_mask&cat3].sort_values(by='members',ascending=False)['group_name'].head(2).values):
  print (ind+9, i)

Your top 10 recommendations based on the above chosen categories are:
1 SQL NYC, The NoSQL & NewSQL Database Meetup
2 Google NYC Tech Talks
3 The Product Group
4 Data Driven NYC (a FirstMark Event)
5 Girl Develop It NYC
6 CoInvent NY (a.k.a NYEBN)
7 New York Entrepreneurs & Startup Network (NYESN)
8 The NYC Business Networking Group (NYCBNG): Award Winner!
9 Free NYC! Free Things To Do In NYC + beyond! (or very cheap)
10 Alternative to the Bar Fun Things To Do In NYC!

###### Part 2: The below code from this point is our take on the KNN model to recommend similar categories to the user from the ones that he already has.
###### This is not something that meetup has implemented as a straight up option and thus we feel this can be a huge value add.

In [36]:
# df_members
df_category = pd.merge(df_members, df_groups,how='left', on=('group_id','city'))
df_category.head()

,member_id,city,group_id,category_id,group_name,members,rating,category_name
0,3,New York,490552,2.0,NYC Customer Makers & Marketers Group,1711.0,4.67,Career & Business
1,3,New York,1474611,34.0,The NYC Gotham Developers Group,5846.0,4.54,Tech
2,3,New York,1490492,34.0,NY Tech Mixer,4628.0,4.28,Tech
3,3,New York,1515830,34.0,The Product Group,15635.0,4.71,Tech
4,3,New York,1574965,34.0,NYC Tech Talks,3207.0,4.57,Tech


In [37]:
df2 = df_category[['member_id','category_name']]
df2.loc[:,('Dummy')]=1

/databricks/python/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 self.obj[key] = _infer_fill_value(value)
/databricks/python/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 self.obj[item] = s

In [38]:
df=df2.pivot_table(index=['member_id'], 
            columns=['category_name'],values='Dummy').fillna(0)
df.head()

category_name,Arts & Culture,Book Clubs,Career & Business,Cars & Motorcycles,Community & Environment,Dancing,Education & Learning,Fashion & Beauty,Fitness,Food & Drink,Games,Health & Wellbeing,Hobbies & Crafts,LGBT,Language & Ethnic Identity,Lifestyle,Movements & Politics,Movies & Film,Music,New Age & Spirituality,Outdoors & Adventure,Paranormal,Parents & Family,Pets & Animals,Photography,Religion & Beliefs,Sci-Fi & Fantasy,Singles,Socializing,Sports & Recreation,Support,Tech,Writing
member_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
65,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
82,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [39]:
interest=df.ix[6,0:] # this is the member_id for which we are identifying the k closest neighbors to match preference
Frame= df.ix[6:,0:] # The other member information to build the knn model on

/local_disk0/tmp/1575564302376-0/PythonShell.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
 from __future__ import absolute_import
/local_disk0/tmp/1575564302376-0/PythonShell.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
 from __future__ import print_function

#### We have attached the code for our KNN model below, this model will help recommend 5 new categories to new users based on their initial choice of 3 categories. The reason this is commented is that this takes 3-4 hours to compile on the 0.88 cores offered by the databricks community edition. This has been tested on jupyter notebooks and it works perfectly.

In [41]:
# The below code snippet fits the knn model based on the other member data, this can be done on a daily once or weekly basis by the Meetup dev team.

import sklearn
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=5).fit(Frame)

In [42]:
# This code snippet enables getting the k nearest neighbors for the data passed in the interest variable

Near=(nbrs.kneighbors([interest]))
Near

x=list(Near[1:])[0][0][1:5]

a=list()
for y in x:
    a.append(df.index.values[y])
a

The below code will print out the final recommendations based on the closest 5 members groups

Suggest=list()
for z in a:
  print(df[df['member_id']==z]['group_name'])